# Bài tập lập trình: Thực hiện xây dựng mạng Neural Network
```
ProtonX - TensorFlow Class
```

### Hướng dẫn làm bài
- Trong bài tập này bạn sẽ sử dụng Python 3.
- Sau khi bạn viết Code của mình xong, hãy chạy dòng Code đó để xem kết quả bên dưới.

### [Quan trọng] Chú ý
- **Không sử dụng hàm `input()` tại bất kỳ dòng lệnh nào**
- **Không thay đổi dòng code return của hàm**

Các bạn sẽ thực hiện `code` trong các phần hiển thị `#TODO: Lập trình tại đây` và thay thế các vị trí `None`. Có những câu hỏi chỉ cần trả về đáp án.

Sau khi viết xong Code của bạn, bạn hãy ấn "SHIFT"+"ENTER" để thực hiện chạy lệnh của Cell đó.

---
Điểm số:
* 10 điểm / Câu

Tiêu chí chấm điểm:
* Các bài tập sẽ được chấm dựa trên các Test-case.
* Các bạn không khởi tạo lại giá trị đầu vào bên trong hàm. Có thể khởi tạo các giá trị này ngoài hàm nhằm mục đích kiểm thử.

# Thực hiện xây dựng mạng Neural Network

Trong tuần này, bạn sẽ thực hiện xây dựng Neural Network đơn giản với:
- 2 lớp ẩn với hàm phi tuyến Relu
- một lớp phân loại softmax

Bạn có thể mở rộng notebook này để có mạng nhiều lớp hơn.

## 1 - Khai báo một số hàm bổ sung

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

## 2 - Quá trình thực hiện

Quá trình toàn bộ mô hình sẽ theo các bước sau

- Khởi tạo các tham số W, b.
- Triển khai `Forward Propagation Module` (lan truyền thuận). Như đường màu tím trong hình bên dưới.
- Tính giá trị mất mát cost.
- Thực hiện `Backward Propagation Module` (lan truyền ngược). Như đường màu xanh trong hình bên dưới.
- Cập nhật các tham số.

<img src="https://storage.googleapis.com/protonx-cloud-storage/images/Feedforward.PNG" style="width:800px;height:500px;">
<caption><center> Ảnh 1</center></caption><br>

**Lưu ý**: với mỗi forward function, sẽ có backward function tương ứng. Đó là lý do mà ở mỗi forward module chúng ta sẽ lưu lại các giá trị này trong `cache` để thuận tiện cho việc tính `gradients`.


## 3 - Khởi tạo


**Hướng dẫn**:
- Khởi tạo giá trị random cho ma trận. Sử dụng `np.random.randn(shape)*0.01`.
- Khởi tạo `0` cho bias. Sử dụng `np.zeros(shape)`.

Các bộ tham số giữa các lớp sẽ có chiều như sau:

- Lớp 1:
  - $\textbf{W}^{(1)} \in \mathbf{R}^{d^{(1)} \times  n } $
  - $b^{(1)} \in \mathbf{R}^{ d^{(1)} \times 1 }$
- Lớp 2:
  - $\textbf{W}^{(2)} \in \mathbf{R}^{d^{(2)} \times d^{(1)}} $
  - $b^{(2)} \in \mathbf{R}^{ d^{(2)} \times 1 }$
- Lớp 3:
  - $\textbf{W}^{(3)} \in \mathbf{R}^{c \times d^{(2)}} $
  - $b^{(3)} \in \mathbf{R}^{ c \times 1 }$

Biến `num_classes` đại diện cho số nhãn $c$ trong trường hợp này.

In [ ]:
def initialize_parameters(n, d_1, d_2, num_classes):
    """
    Hàm khởi tạo tham số ban đầu
    Đầu vào:
      n: int
        Chiều của x đầu vào
      d_1: int
        Chiều của lớp ẩn 1
      d_2:
        Chiều của lớp ẩn 2
      num_classes:
        Số lượng nhãn
    Đầu ra:
      parameters: python dictionary
      Bao gồm:
        W1: ma trận W1 lớp 1 với chiều (d_1, n)
        b1: vector bias b1 lớp 1 với chiều (d_1, 1)
        W2: ma trận W2 lớp 2 với chiều (d_2, d_1)
        b2: vector bias b2 lớp 2 với chiều (d_2, 1)
        W3:  ma trận W3 lớp 3 với chiều (num_classes, d_2)
        b3: vector bias b3 lớp 3 với chiều (num_classes, 1)
    """

    # Khởi tạo tham số

    np.random.seed(42)
    W1 = np.random.randn(d_1, n) * 0.01
    b1 = np.zeros((d_1, 1))
    W2 = np.random.randn(d_2, d_1) * 0.01
    b2 = np.zeros((d_2, 1))
    W3 = np.random.randn(num_classes, d_2) * 0.01
    b3 = np.zeros((num_classes, 1))

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,
                  "W3": W3,
                  "b3": b3,
                  }

    return parameters

In [ ]:
parameters = initialize_parameters(784, 256, 128, 10)
print("W1.shape = {}".format(parameters["W1"].shape))
print("b1.shape = {}".format(parameters["b1"].shape))
print("W2.shape = {}".format(parameters["W2"].shape))
print("b2.shape = {}".format(parameters["b2"].shape))
print("W3.shape = {}".format(parameters["W3"].shape))
print("b3.shape = {}".format(parameters["b3"].shape))

W1.shape = (256, 784)
b1.shape = (256, 1)
W2.shape = (128, 256)
b2.shape = (128, 1)
W3.shape = (10, 128)
b3.shape = (10, 1)


## 4 - Lan truyền thuận (Forward propagation module)



### 4.1 - Lan truyền thuận tuyến tính (Linear Forward)

Sau khi khởi tạo các tham số cần thiết, bây giờ bạn sẽ thực hiện forward propagation module. Bạn sẽ thực hiện theo thứ tự sau:

- Tuyến tính (Linear)
- Tuyến tính $\rightarrow $ Phi tuyến (Linear $\rightarrow $ Activation)


#### 4.1.1. Trên 1 điểm dữ liệu


Công thức lan truyền thuận tuyến tính như sau:

$$\textbf{z}^{(l)} = \textbf{W}^{(l)}\textbf{a}^{(l-1)} +\textbf{b}^{(l)}\tag{4}$$

trong đó $\textbf{a}^{(0)} = \textbf{x}$.


#### 4.1.2. Trên toàn tập dữ liệu


Công thức Linear Forward như sau:



$$\textbf{Z}^{(l)} = \textbf{W}^{(l)}\textbf{A}^{(l-1)} +\textbf{b}^{(l)}\tag{5}$$


trong đó $\textbf{A}^{(0)} = \textbf{X}$




In [ ]:
def linear_forward(A_pre, W, b):
    """
    Tiến hành lan truyền thuận tuyến tính (linear forward)
    Đầu vào:
      A_pre:
        Dạng: numpy array
        Miêu tả: Đầu ra của lớp phía trước trên toàn bộ tập dữ liệu
        Chiều: (Chiều của lớp phía trước, Số lượng điểm dữ liệu)
      W:
        Dạng: numpy array
        Miêu tả: Ma trận tham số của lớp hiện tại
        Chiều: (Chiều của lớp hiện tại, Chiều của lớp phía trước)
      b:
        Dạng: numpy array
        Miêu tả: Vector bias của lớp hiện tại
        Chiều: (Chiều của lớp hiện tại, 1)
    Đầu ra:
      Z:
        Dạng: numpy array
        Miêu tả: Đầu ra của phép nhân tuyến tính và cũng là đầu vào của hàm activation
        Chiều: (Chiều của lớp hiện tại, Số lượng điểm dữ liệu)
      cache:
        Dạng: python tuple
        Miêu tả: Bao gồm các biến A_pre, W và b
    """

    Z = np.dot(W, A_pre) + b

    cache = (A_pre, W, b)

    return Z, cache

Test code

In [ ]:
try:
  A1 = np.ones((256, 60000))
  W2 = np.random.normal(size=((128, 256)))
  b2 = np.ones(((128, 1)))

  Z, linear_cache = linear_forward(A1, W2, b2)
  print("Z.shape: {}".format(Z.shape))
except Exception as e:
  print("Lỗi thực thi: {}", e)

Z.shape: (128, 60000)


### 4.2. Lan truyền thuận phi tuyến (Activation Forward)

#### 4.2.1 Công thức RELU

$$f(z) = \left\{\begin{matrix}
0 \ \forall z \leq 0 \\
z \ \forall z > 0 \\
\end{matrix}\right. = \max\left \{0, z  \right \} = z\textbf{1}_{z>1}$$

**Chú ý:** Các phép tính trong bài toán này đều sử dụng ma trận cho thuật toán Gradient Descent thay vì vector trong bài tập Softmax

In [ ]:
def relu(Z):
    """
    Thực hiện hàm RELU trên ma trận Z
    Đầu vào:
      Z:
        Dạng: numpy array
        Miêu tả: Ma trận Z. Đầu ra của phép nhân tuyến tính
        Chiều: (Chiều của lớp hiện tại, Số lượng điểm dữ liệu)
        Ví dụ: (256, 60000)
    Đầu ra:
      A:
        Dạng: numpy array
        Miêu tả: Giá trị A sau khi đưa Z qua Relu
        Chiều: (Chiều của lớp hiện tại, 60000)
        Ví dụ: (256, 60000)
      cache:
        Dạng: numpy array
        Miêu tả:
          Ma trận Z. Đầu ra của phép nhân tuyến tính (hay đầu vào của phép phi tuyến)
          Việc lưu trữ này để sử dụng cho thuật toán lan truyền ngược
        Chiều: (Chiều của lớp hiện tại, Số lượng điểm dữ liệu)
        Ví dụ: (256, 60000)
    """

    # Lập trình tại đây

    A = np.maximum(0,Z)

    cache = Z

    return A, cache

#### 4.2.2. Công thức Softmax



Trong trường hợp này ta sẽ lấy softmax trên từng cột của $\textbf{Z}$ với $\textbf{Z} \in \mathbf{R}^{c \times m} $

Với:

- $c$: Số nhãn
- $m$: Số lượng điễm dữ liệu


$$\hat{\textbf{Y}} = \text{softmax}(\textbf{Z}) =  \begin{bmatrix}
\text{softmax}(\begin{bmatrix}
z_1^1 \\
\ \vdots \\
z_c^1 \\
\end{bmatrix}) & \text{softmax}(\begin{bmatrix}
z_1^2 \\
\ \vdots \\
z_c^2 \\
\end{bmatrix}) & \dots & \text{softmax}(\begin{bmatrix}
z_1^m \\
\ \vdots \\
z_c^m \\
\end{bmatrix})
\end{bmatrix} $$

In [ ]:
def softmax(Z):
  """
  Thực hiện hàm softmax trên ma trận Z
  Đầu vào:
    Z:
      Dạng: numpy array
      Miêu tả: Ma trận Z. Đầu ra của phép nhân tuyến tính
      Chiều: (Số lượng nhãn, Số lượng điểm dữ liệu)
      Ví dụ: (10, 60000)
  Đầu ra:
    Y_hat:
      Dạng: numpy array
      Miêu tả: Chuẩn hóa ma trận Z theo phân phối softmax có tổng các giá trị bằng 1
      Chiều: (Số lượng nhãn, Số lượng điểm dữ liệu)
      Ví dụ: (10, 60000)
    cache:
      Dạng: numpy array
      Miêu tả:
        Ma trận Z. Đầu ra của phép nhân tuyến tính
        Việc lưu trữ này để sử dụng cho thuật toán lan truyền ngược
      Chiều: (Số lượng nhãn, Số lượng điểm dữ liệu)
      Ví dụ: (10, 60000)
  """

  e_Z = np.exp(Z)

    # Lay tung dong data / tong so
  Y_hat = e_Z/ e_Z.sum(axis=0) # + theo row

  cache = Z

  return Y_hat, cache

Test code


Kiểm tra lại các cột sau khi sử dụng Softmax có cho kết quả tổng bằng 1 hay không.

In [ ]:
Z = np.ones(((10, 60000)))
Y_hat, _ = softmax(Z)


np.sum(Y_hat[:,3])

1.0

### 4.3 - Lan truyền thuận trên một lớp (Linear-Activation Forward)

Trong notebook này, bạn sẽ sử dụng activation function `RELU`:

<!-- - **Sigmoid**: $ \text{Sigmoid}(\textbf{Z}) = \sigma(\textbf{Z}) = \sigma(\textbf{W} \textbf{A} + \textbf{b}) = \frac{1}{ 1 + e^{-(\textbf{W} \textbf{A} + \textbf{b})}}$. Function này sẽ trả về 2 giá trị "`A`" (giá trị activation) và "`cache`" (chứa giá trị của `Z`) - Các giá trị này sẽ được sử dụng cho Backward Function. Để sử dụng, bạn chỉ cần gọi theo mẫu sau:
```python
A, activation_cache = sigmoid(Z)
``` -->

- **ReLU**: công thức toán của ReLU là $\textbf{A} = \text{Relu}(\textbf{Z}) = \text{max}(0, \textbf{Z})$.  Function này sẽ trả về 2 giá trị "`a`" (giá trị activation) và "`cache`" (chứa giá trị của `Z`) - Các giá trị này sẽ được sử dụng cho Backward Function. Để sử dụng, bạn chỉ cần gọi theo mẫu sau:
``` python
A, activation_cache = relu(Z)
```

- Lớp 1:

$$\textbf{A}^{(1)} = \text{Relu}(\textbf{Z}^{(1)}) $$

- Lớp 2:

$$\textbf{A}^{(2)} = \text{Relu}(\textbf{Z}^{(2)}) $$

- Lớp 3 (Lớp cuối):

$$\textbf{A}^{(3)} = \hat{\textbf{Y}} = \text{Softmax}(\textbf{Z}^{(3)}) \tag{6}$$

In [ ]:
def linear_activation_forward(A_prev, W, b, activation):
    """
    Tiến hành non-linear forward
    A_prev, W và B đưa qua hàm linear_forward thu được Z
    Từ đó Z sẽ được đưa qua hàm phi tuyến RELU
    Đầu vào:
      A_prev:
        Dạng: numpy array
          Miêu tả: Đầu ra của lớp phía trước trên toàn bộ tập dữ liệu
          Chiều: (Chiều của lớp phía trước, Số lượng điểm dữ liệu)
      W:
        Dạng: numpy array
        Miêu tả: Ma trận tham số của lớp hiện tại
        Chiều: (Chiều của lớp hiện tại, Chiều của lớp phía trước)
      b:
        Dạng: numpy array
        Miêu tả: Vector bias của lớp hiện tại
        Chiều: (Chiều của lớp hiện tại, 1)
      activation:
        Dạng: python string
        Miêu tả: Tên hàm activation. Ví dụ "softmax" or "relu"

    Returns:
      A:
        Dạng: numpy array
        Miêu tả: Đầu ra của phép phi tuyến hay đầu ra của hàm activation function
        Chiều: (Chiều của lớp hiện tại, Số lượng điểm dữ liệu)
      cache:
        Dạng: python tuple: (linear_cache, activation_cache)
        Bao gồm:
          linear_cache: tuple (A_pre, W, b) đầu vào của phép tuyến tính được lưu lại
          activation_cache: Z đầu vào của phép phi tuyến được lưu lại
    """
    # Ta sẽ phân loại việc biến đổi tuyến tính tùy theo đầu vào activation
    if activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
    elif activation == "softmax":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = softmax(Z)

    cache = (linear_cache, activation_cache)

    return A, cache

Test code

In [ ]:
try:
  A2 = np.ones((256, 60000))
  W3 = np.random.normal(size=((10, 256)))
  b3 = np.ones(((10, 1)))

  A, cache = linear_activation_forward(A2, W3, b3, activation='softmax')
  print("A.shape: {}".format(A.shape))

  # cache là một tuple

  linear_cache, activation_cache = cache
  A2, W3, b3 = linear_cache

  print("activation_cache.shape hay Z.shape: {}".format(activation_cache.shape))
  print("linear_cache bao gồm A2 với chiều A2.shape: {}, W3 với chiều W3.shape: {}, b3 với chiều b3.shape: {}".format(A2.shape, W3.shape, b3.shape))
except Exception as e:
  print("Lỗi thực thi: ", e)

A.shape: (10, 60000)
activation_cache.shape hay Z.shape: (10, 60000)
linear_cache bao gồm A2 với chiều A2.shape: (256, 60000), W3 với chiều W3.shape: (10, 256), b3 với chiều b3.shape: (10, 1)


## 5 - Hàm mất mát (Cost function)

Công thức toán của cross-entropy cost $J$: $$J(\textbf{W}) = -\frac{1}{m} \sum\limits_{i = 1}^{m} \sum\limits_{j = 1}^{c} \textbf{y}^{(i)}\log (\hat{\textbf{y}}^{(i)})\tag{7}$$

Với
- $c$: số nhãn
- $m$: số điểm dữ liệu

Chú ý trong bài này đầu vào của hàm là theo toàn bộ tập dữ liệu.


**TODO 1**: Lập trình công thức hàm Cost Function

In [ ]:
def compute_cost(Y_hat, Y):
    """
    Tiến hành categorical cross-entropy trên toàn bộ tập dữ liệu
    Đầu vào:
    Y_hat:
      Dạng: numpy array
      Miêu tả: Đầu ra của lớp softmax trên toàn bộ tập dữ liệu
      Chiều: (Số lượng nhãn, Số lượng điểm dữ liệu)
    Y:
      Dạng: numpy array
      Miêu tả: Nhãn của dữ liệu (Dạng One hot)
      Chiều: (Số lượng nhãn, Số lượng điểm dữ liệu)

    Returns:
      Dạng: numpy float
      Miêu tả: Giá trị mất mát trên toàn bộ tập dữ liệu
    """

    # Nhân Hadamard và lấy tổng theo cột.
    # Lấy tổng theo cột vì chiều đầu vào có số dòng là số lượng nhãn cho nên ta
    # cộng tất cả giá trị trong cùng 1 cột lại với nhau để tìm giá trị mất mát
    # trên 1 điểm dữ liệu đó

    # Lập trình tại đây

    m = Y.shape[1]

    cost = 0

    # Lấy tổng theo theo cột kèm theo chia trung bình
    cost -= np.sum(Y * np.log(Y_hat))

    return cost/m

Test code

In [ ]:
Y =  np.array(
      [[0., 0.],
       [0., 0.],
       [0., 1.],
       [0., 0.],
       [0., 0.],
       [1., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

Y_hat =  np.array(
      [[0.1, 0.05],
       [0.1, 0.05],
       [0.1, 0.05],
       [0.2, 0.2],
       [0.05, 0.05],
       [0.2, 0.1],
       [0.1, 0.1],
       [0.05, 0.2],
       [0.05, 0.1],
       [0.05, 0.1]])

print("cost = {}".format(compute_cost(Y_hat, Y)))
# result = Y*np.log(Y_hat)

# print(np.sum(result)/2)
# print(np.log(Y_hat)) 2x10 @10x2  > 2x2

cost = 2.3025850929940455


**Kết quả mong đợi**:

```
cost = 2.3025850929940455
```


Giá trị này tương đương với $-(\text{log}(0.2) + \text{log}(0.05))$

## 6 - Lan truyền ngược (Backward propagation module)

Cũng giống như lan truyền thuận (forward propagation), bây giờ bạn sẽ thực hiện các function cho backpropagation. Chú ý lan truyền thuận được sử dụng để tính toán gradient của hàm mất mát với các tham số (parameters).


Mỗi một hàm lan truyền thuận phía trên sẽ đi kèm một hàm lan truyền ngược

### 6.1 - Lan truyền ngược tuyến tính (Linear backward)



<img src="https://storage.googleapis.com/protonx-cloud-storage/images/Backprop3.PNG" style="width:250px;height:300px;">
<caption><center>Hình 4</center></caption>

Tại lớp thứ $l$, ta có: $\textbf{Z}^{(l)} = \textbf{W}^{(l)} \textbf{A}^{(l-1)} + \textbf{b}^{(l)}$ (Activation tương ứng).

Nhiệm vụ tiên quyết là tính được các giá trị $\textbf{E}^{(l)}$ vì ta có thể dùng $\textbf{E}^{(l)}$ để tính ra được $\textbf{E}^{(l-1)}$

Giá sử bạn đã tính đạo hàm của $\textbf{E}^{(l)} = d\textbf{Z}^{(l)} = \frac{\partial J }{\partial \textbf{Z}^{(l)}}$.



Bạn muốn tính $(d\textbf{W}^{(l)}, d\textbf{b}^{(l)}, d\textbf{A}^{(l-1)})$.

3 giá trị $(d\textbf{W}^{(l)}, d\textbf{b}^{(l)}, d\textbf{A}^{(l-1)}$ được tính toán từ $d\textbf{Z}^{(l)}$ thông qua các công thức sau:


- Đạo hàm của hàm mất mát trên $\textbf{W}^{(l)}$

$$
d\textbf{W}^{(l)} = \frac{\partial {J} }{\partial \textbf{W}^{(l)}} = \frac{1}{m} d\textbf{Z}^{(l)} \textbf{A}^{(l-1) T} = \frac{1}{m} \textbf{E}^{(l)} \textbf{A}^{(l-1) T} \tag{8}
$$

- Đạo hàm của hàm mất mát trên $\textbf{b}^{(l)}$.Một cách cụ thể thì đây chính là trung bình tổng các cột của ma trận ${E}^{(l)}$


$$ d\textbf{b}^{(l)} = \frac{\partial J }{\partial \textbf{b}^{(l)}} = \frac{1}{m} \sum_{i = 1}^{m} d\textbf{Z}^{(l)(i)} = \frac{1}{m} \sum_{i = 1}^{m} \textbf{E}^{(l)(i)} \tag{9}$$

      
- Đạo hàm của hàm mất mát trên $\textbf{A}^{(l-1)}$

$$ d\textbf{A}^{(l-1)} = \frac{\partial J }{\partial \textbf{A}^{(l-1)}} = \textbf{W}^{(l) T} d\textbf{Z}^{(l)} = \textbf{W}^{(l) T} \textbf{E}^{(l)} \tag{10}$$




**TODO 2:** Lập trình các công thức trên cho hàm lan truyền ngược tuyến tính

In [ ]:
def linear_backward(E, cache):
    """
    Tính toán đạo hàm của hàm mất mát trên A, W và b
    Đầu vào:
      E hoặc dZ:
        Dạng: numpy array
        Miêu tả: Đạo hàm của hàm mất mát với đầu ra của phép tuyến tính tại lớp hiện tại (l)
        Chiều: (Chiều của lớp hiện tại, Số lượng điểm dữ liệu)
      cache
        Dạng: Python tuple
        Miêu tả: Biến lưu trữ (cache) của lớp linear
        Bao gồm các biến A_prev, W và b
    Đầu ra:
      dA_prev:
        Dạng: numpy array
        Miêu tả: Đạo hàm của hàm mất mát với đầu ra phép phi tuyến của lớp l-1
        Chiều: (Chiều của lớp phía trước, Số lượng điểm dữ liệu)
      dW:
        Dạng: numpy array
        Miêu tả: Đạo hàm của hàm mất mát với tham số của lớp hiện tại (l)
        Chiều: (Chiều của lớp hiện tại, Chiều của lớp phía trước)
      db:
        Dạng: numpy array
        Miêu tả: Đạo hàm của hàm mất mát với bias của lớp hiện tại (l)
        Chiều: (Chiều của lớp hiện tại, 1)
    """
    # Lập trình tại đây
    A_prev, W, b = cache

    # Tính gradient trên W
    m = E.shape[1]
    dW = (1/m)*E.dot(A_prev.T)

    # Tính gradient trên b
    db = np.expand_dims (np.sum(E,axis = 1)/m, axis = 1)

    # Tính gradient trên A phía trước
    dA_prev = W.T.dot(E)

    return dA_prev, dW, db

In [ ]:
#   E2 = dZ = np.ones((128, 60000))
#   A1 = np.ones((256, 60000))
#   W2 = np.ones((128, 256))
#   b2 = np.ones((128,1))
#   linear_cache = (
#       A1, W2, b2
#   )
#   m = E2.shape[0]
#   (1/m)*E2.dot(A1.T)

#   print(np.sum(E2,axis = 1).shape)

In [ ]:
try:
  # Test trên layer 2
  E2 = dZ = np.ones((128, 60000))
  A1 = np.ones((256, 60000))
  W2 = np.ones((128, 256))
  b2 = np.ones((128,1))
  linear_cache = (
      A1, W2, b2
  )

  dA1, dW2, db2 = linear_backward(E2, linear_cache)
  print('Chiều của dA1: {}'.format(dA1.shape))
  print('Chiều của dW2: {}'.format(dW2.shape))
  print('Chiều của db2: {}'.format(db2.shape))
  print('Giá trị dA1 được tính đúng?: {}'.format(np.array_equal(np.full(A1.shape, fill_value=128.), dA1)))
  print('Giá trị dW2 được tính đúng?: {}'.format(np.array_equal(np.full(W2.shape, fill_value=1.), dW2)))
  print('Giá trị db2 được tính đúng?: {}'.format(np.array_equal(np.full(b2.shape, fill_value=1.), db2)))
except Exception as e:
  print("Lỗi thực thi: ", e)

Chiều của dA1: (256, 60000)
Chiều của dW2: (128, 256)
Chiều của db2: (128, 1)
Giá trị dA1 được tính đúng?: True
Giá trị dW2 được tính đúng?: True
Giá trị db2 được tính đúng?: True


In [ ]:
dW2.shape

(128, 256)

**Kết quả mong đợi**:

```
Chiều của dA1: (256, 60000)
Chiều của dW2: (128, 256)
Chiều của db2: (128, 1)
Giá trị dA1 được tính đúng?: True
Giá trị dW2 được tính đúng?: True
Giá trị db2 được tính đúng?: True
```


### 6.2 - Lan truyền ngược phi tuyến (Activation backward)

**TODO 3**: Lập trình đạo hàm của RELU




Nếu $f(.)$  là activation function `relu_backward` được tính: $$\textbf{E}^{(l)} = d\textbf{Z}^{(l)} = d\textbf{A}^{(l)} * f'(\textbf{Z}^{(l)})$$

- Với **RELU**
$$f'(z) = \left\{\begin{matrix}
0 \ \ \forall z < 0 \\
1 \ \ \forall z > 0 \\
DNE \ \ \text{if} \ \ z = 0 \\
\end{matrix}\right.$$

**RELU** `không có đạo` hàm tại 0 tuy nhiên khi lập trình ta sẽ cài đặt sẵn với trường hơp $z=0$ đạo hàm của RELU sẽ bằng `0`. Vậy công thức trở thành.

$$f'(z) = \left\{\begin{matrix}
0 \ \ \forall z <= 0 \\
1 \ \ \forall z > 0 \\
\end{matrix}\right.$$

Cùng lập trình đạo hàm này với trình tự:

- Tất cả những giá trị của $\textbf{Z}$ nhỏ hơn hoặc bằng 0 sẽ được đặt thành 0
- Tất cả những giá trị của $\textbf{Z}$ lớn hơn 0 sẽ được đặt thành 1

In [ ]:
def relu_derivative(Z):
  """
  Thực hiện đạo hàm của RELU trên ma trận Z
  Đầu vào:
    Z:
      Dạng: numpy array
      Miêu tả: Đầu ra của phép nhân tuyến tính và cũng là đầu vào của hàm activation
      Chiều: (Chiều của lớp hiện tại, Số lượng điểm dữ liệu)
  Đầu ra:
    Z_grad:
      Dạng: numpy array
      Miêu tả: Đạo hàm của hàm RELU với Z
      Chiều: (Chiều của lớp hiện tại, Số lượng điểm dữ liệu)
  """
  # Lập trình tại đây
  # print('start')
  # Z < 0.0
  Z_grad = np.where(Z<0, 0, 1)
  # print('done')

  return Z_grad


In [ ]:
# Z = np.array([2,3])
np.full((128, 60000), 2.5)
result = np.where(Z < 0, 0, 1)
print(result)

[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]


Test code

In [ ]:
try:
  # Test trên layer 2
  Z1 = np.full((128, 60000), 2.)
  Z2 = np.full((128, 60000), -2.)

  Z1_derivative = relu_derivative(Z1)
#   print(Z1_derivative)
  Z2_derivative = relu_derivative(Z2)


  print('Chiều của đạo hàm hàm RELU trên Z1: {}'.format(Z1_derivative.shape))
  print('Chiều của đạo hàm hàm RELU trên Z2: {}'.format(Z2_derivative.shape))
  print('Giá trị đạo hàm hàm RELU trên Z1 đã được tính đúng?: {}'.format(np.array_equal(Z1_derivative, np.ones(Z1.shape))))
  print('Giá trị đạo hàm hàm RELU trên Z2 đã được tính đúng?: {}'.format(np.array_equal(Z2_derivative, np.zeros(Z2.shape))))
except Exception as e:
  print("Lỗi thực thi: ", e)

Chiều của đạo hàm hàm RELU trên Z1: (128, 60000)
Chiều của đạo hàm hàm RELU trên Z2: (128, 60000)
Giá trị đạo hàm hàm RELU trên Z1 đã được tính đúng?: True
Giá trị đạo hàm hàm RELU trên Z2 đã được tính đúng?: True


**Kết quả mong đợi**:

```
Chiều của đạo hàm hàm RELU trên Z1: (128, 60000)
Chiều của đạo hàm hàm RELU trên Z2: (128, 60000)
Giá trị đạo hàm hàm RELU trên Z1 đã được tính đúng?: True
Giá trị đạo hàm hàm RELU trên Z2 đã được tính đúng?: True
```

**TODO 4**: Lập trình hàm lan truyền ngược trên hàm phi tuyến

Cụ thể là đạo hàm của **hàm mất mát** trên giá trị đầu vào hàm phi tuyến hay nói cách khác chính là:

$$\textbf{E}^{(l)} = d\textbf{Z}^{(l)} = \frac{\partial J }{\partial \textbf{Z}^{(l)}} = d\textbf{A}^{(l)} * f'(\textbf{Z}^{(l)})$$

<img src="https://storage.googleapis.com/protonx-cloud-storage/backprop5.PNG">

Cho nên đầu vào của hàm tính giá trị $\textbf{E}^{(l)}$ sẽ bao gồm 2 giá trị:
- Đạo hàm của hàm mất mát với đầu ra phép phi tuyến ở lớp hiện tại đã được tính trước: $d\textbf{A}^{(l)}$
- Giá trị $\textbf{Z}$ đã lưu lại khi thực hiện phi tuyến theo chiều thuận

Hàm này sẽ thực hiện nhiệm vụ:

- Sử dụng hàm `relu_derivative` để tính đạo hàm $f'(\textbf{Z}^{(l)})$ của hàm RELU với $\textbf{Z}$
- Sau đó lấy đầu vào $d\textbf{A}^{(l)}$ nhân Hadamard với giá trị tìm được

In [ ]:
def relu_backward(dA, cache):
    """
    Thực hiện lan truyền ngược qua hàm phi tuyến RELU
    Hay nói cách khác đạo hàm của hàm mất mát trên Z
    Đầu vào:
      dA:
        Dạng: numpy array
        Miêu tả: Đạo hàm của hàm mất mát với đầu ra phép phi tuyến của lớp l
        Chiều: (Chiều của lớp hiện tại, Số lượng điểm dữ liệu)
      cache:
        Dạng: numpy array
        Miêu tả: Z được lưu lại từ hàm linear-activation forward
        Chiều: (Chiều của lớp hiện tại, Số lượng điểm dữ liệu)
    Đầu ra:
      E (dZ):
        Dạng: numpy array
        Miêu tả: Đạo hàm của giá trị mất mát với Z
        Chiều: (Chiều của lớp hiện tại, Số lượng điểm dữ liệu)
    """
    # Lập trình tại đây
    Z = cache
    Z_grad = relu_derivative(Z)
    E = dA * Z_grad

    return E

Test Code

In [ ]:
try:
  dA1 = np.full((128, 60000), -2.)
  cache1 = np.full((128, 60000), -3.)
  E1 = relu_backward(dA1, cache1)

  dA2 = np.full((128, 60000), 1.)
  cache2 = np.full((128, 60000), 1.)
  E2 = relu_backward(dA2, cache2)

  print('Chiều của E1: {}'.format(E1.shape))
  print('Chiều của E2: {}'.format(E2.shape))
  print('Giá trị E1 đã được tính đúng?: {}'.format(np.array_equal(E1, np.full(E1.shape, 0.))))
  print('Giá trị E2 đã được tính đúng?: {}'.format(np.array_equal(E2, np.full(E2.shape, 1.))))
except Exception as e:
  print("Lỗi thực thi: ", e)

Chiều của E1: (128, 60000)
Chiều của E2: (128, 60000)
Giá trị E1 đã được tính đúng?: True
Giá trị E2 đã được tính đúng?: True


**Kết quả mong đợi**:

```
Chiều của E1: (128, 60000)
Chiều của E2: (128, 60000)
Giá trị E1 đã được tính đúng?: True
Giá trị E2 đã được tính đúng?: True
```

### 6.3 Lan truyền trên một lớp (Linear-Activation backward)

**TODO 5:** Kết hợp phần `6.1` và `6.2` để tạo thành một hàm đầy đủ việc lan truyền ngược trên một lớp ẩn bất kỳ. Chú ý **đường màu xanh**.

<img src="https://storage.googleapis.com/protonx-cloud-storage/Backprop7.PNG" >

In [ ]:
def linear_activation_backward(dA, cache, activation):
    """
    Thực hiện lan truyền ngược trên lớp này
    Sử dụng kết hợp relu_backward và hàm linear_backward
    Đầu vào:
      dA:
        Dạng: numpy array
        Miêu tả: Đạo hàm của hàm mất mát với đầu ra phép phi tuyến của lớp l
        Chiều: (Chiều của lớp hiện tại, Số lượng điểm dữ liệu)
      cache:
        Dạng: Python tuple
        Miêu tả: (linear_cache, activation_cache)
        Bao gồm:
          linear_cache:
            Dạng: Python tuple
            Miêu tả: cache lưu lại từ phép tuyến tính thuận: (A_pre, W, b)
          activation_cache:
            Dạng: Numpy array
            Miêu tả: cache lưu lại từ phép phi tính thuận: (Z)
      activation:
        Dạng: Python string
        Miêu tả: Tên của activation
        Ví dụ: "relu"
    Đầu ra:
      dA_prev:
        Dạng: numpy array
        Miêu tả: Đạo hàm của hàm mất mát với đầu ra phép phi tuyến của lớp l-1
        Chiều: (Chiều của lớp phía trước, Số lượng điểm dữ liệu)
      dW:
        Dạng: numpy array
        Miêu tả: Đạo hàm của hàm mất mát với tham số của lớp hiện tại (l)
        Chiều: (Chiều của lớp hiện tại, Chiều của lớp phía trước)
      db:
        Dạng: numpy array
        Miêu tả: Đạo hàm của hàm mất mát với bias của lớp hiện tại (l)
        Chiều: (Chiều của lớp hiện tại, 1)
    """
    # Lập trình tại đây

    linear_cache, activation_cache = cache

    # Thực hiện activation backward
    # Từ dA và activation_cache để tính ra dZ
    linear_cache, activation_cache = cache
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        # print('done')

    # Hàm này có thể thiết kế để sử dụng nhiều hàm khác nhau trong tương lai
    # Bạn không cần code elif này
    elif activation == "gelu":
        pass

    # Thực hiện linear backward
    # Từ dZ và linear_cache để tính ra dA_prev, dW và db
    dA_prev, dW, db = linear_backward(dZ, linear_cache)

    return dA_prev, dW, db

Test code

In [ ]:
try:
  np.random.seed(42)
  dA2 = np.full((128, 60000), 4.)
  activation = 'relu'
  A1 = np.ones((256, 60000))
  W2 = np.full((128, 256), 2.)
  b2 = np.ones((128,1))
  linear_cache = (
      A1, W2, b2
  )
  activation_cache = Z = np.full((128, 60000), 2.5)
  cache = (linear_cache, activation_cache)

  dA1, dW, db = linear_activation_backward(dA2, cache, activation)


  print('Chiều của dA1: {}'.format(dA1.shape))
  print('Chiều của dW: {}'.format(dW.shape))
  print('Chiều của db: {}'.format(db.shape))
  print('Giá trị dA1 đã được tính đúng?: {}'.format(np.array_equal(dA1, np.full(dA1.shape, 1024.))))
  print('Giá trị dW đã được tính đúng?: {}'.format(np.array_equal(dW, np.full(W2.shape, 4.))))
  print('Giá trị db đã được tính đúng?: {}'.format(np.array_equal(E2, np.full(E2.shape, 1.))))
except Exception as e:
  print("Lỗi thực thi: ", e)

Chiều của dA1: (256, 60000)
Chiều của dW: (128, 256)
Chiều của db: (128, 1)
Giá trị dA1 đã được tính đúng?: True
Giá trị dW đã được tính đúng?: True
Giá trị db đã được tính đúng?: True


In [ ]:
# relu_backward(dA2, activation_cache)

**Kết quả mong đợi**:

```
Chiều của dA1: (256, 60000)
Chiều của dW: (128, 256)
Chiều của db: (128, 1)
Giá trị dA1 đã được tính đúng?: True
Giá trị dW đã được tính đúng?: True
Giá trị db đã được tính đúng?: True
```

### 6.4 - Cập nhật tham số

Trong phần này, bạn sẽ thực hiện cập nhật các tham số:


$$ \textbf{W}^{(l)} = \textbf{W}^{(l)} - \alpha \text{ } d\textbf{W}^{(l)} \tag{16}$$
$$ \textbf{b}^{(l)} = \textbf{b}^{(l)} - \alpha \text{ } d\textbf{b}^{(l)} \tag{17}$$

trong đó $\alpha$ là learning rate. Sau khi tính cập nhật giá trị Parameters, chúng ta sẽ lưu trữ trong trong 1 parameters dictionary.

**TODO 6**: Thực hiện `update_parameters()`.

**Hướng dẫn**: Cập nhật tham số sử dụng Gradient Descent cho $\textbf{W}^{(l)}$ và $\textbf{b}^{(l)}$ của từng layer $l = 1, 2, ..., L$.


In [ ]:
def update_parameters(parameters, grads, learning_rate):
    """
    Lặp qua các lớp và cập nhật tham số sử dụng Gradient Descent
    Đầu vào:
      parameters:
        Dạng: Python dictionary
        Miêu tả: Chứa các tham số với key là tên và giá trị là numpy array tham số đó
        Ví dụ: {
          W1: ...,
          b1: ...,
          ...
        }
      grads:
        Dạng: Python dictionary
        Miêu tả: Chứa gradient của các tham số với key là tên gradient và giá trị là numpy array của gradient
        Ví dụ: {
          dW1: ...,
          db1: ...,
          ...
        }
    Đầu ra:
      parameters:
        Dạng: Python dictionary
        Miêu tả: Chứa các tham số đã được cập nhật gradient
        Ví dụ: {
          W1: ...,
          b1: ...,
          ...
        }
    """
    # Lập trình tại đây
    W1 = parameters['W1'] - learning_rate * grads['dW1']
    b1 = parameters['b1'] - learning_rate * grads['db1']

    W2 = parameters['W2'] - learning_rate * grads['dW2']
    b2 = parameters['b2'] - learning_rate * grads['db2']

    W3 = parameters['W3'] - learning_rate * grads['dW3']
    b3 = parameters['b3'] - learning_rate * grads['db3']

    parameters = {'W1': W1, 'b1': b1, 'W2': W2, 'b2':b2, 'W3': W3, 'b3':b3}

    return parameters

Test code

In [ ]:
try:
  n = 784
  d_1 = 128
  d_2 = 256
  d_3 = 1028
  np.random.seed(42)
  parameters = {
      "W1": np.random.randn(d_1, n),
      "b1": np.zeros((d_1, 1)),
      "W2": np.random.randn(d_2, d_1),
      "b2": np.zeros((d_2, 1)),
      "W3": np.random.randn(d_3, d_2),
      "b3": np.zeros((d_3, 1)),
  }

  grads = {
      "dW1": np.full((d_1, n), -0.02),
      "db1": np.full((d_1, 1), 0.1),
      "dW2": np.full((d_2, d_1), -0.4),
      "db2": np.full((d_2, 1), 0.5),
      "dW3": np.full((d_3, d_2), 0.6),
      "db3": np.full((d_3, 1), -0.02),
  }

  lr = 0.01
  updated_parameters = update_parameters(parameters, grads, lr)
  exp = {'W1': 106.015, 'b1': -0.128, 'W2': 200.643, 'b2': -1.28, 'W3': -1661.189, 'b3': 0.206}
  for key in updated_parameters:
    print("{} được cập nhật đúng?: {}".format(key, exp[key] == np.round(np.sum(updated_parameters[key]), 3)))
except Exception as e:
  print("Lỗi thực thi: ", e)

# print(updated_parameters)

W1 được cập nhật đúng?: True
b1 được cập nhật đúng?: True
W2 được cập nhật đúng?: True
b2 được cập nhật đúng?: True
W3 được cập nhật đúng?: True
b3 được cập nhật đúng?: True


**Kết quả mong đợi**:

```
W1 được cập nhật đúng?: True
b1 được cập nhật đúng?: True
W2 được cập nhật đúng?: True
b2 được cập nhật đúng?: True
W3 được cập nhật đúng?: True
b3 được cập nhật đúng?: True
```

##  7 - Train Model

### 7.1. Tải dữ liệu MNIST

In [ ]:
import tensorflow as tf
(X_train, Y_train), (X_val, Y_val) = tf.keras.datasets.mnist.load_data()

Một số thông tin quan trọng

In [ ]:
# Số lượng nhãn
num_classes = 10

num_of_train_images, width, height = X_train.shape

# Chiều ảnh được duỗi
image_vector_size = width * height


print("""
  Số lượng ảnh train: {}
  Chiều dài ảnh train: {}
  Chiều cao ảnh train: {}
  Chiều ảnh được duỗi: {}
""".format(num_of_train_images, width, height, image_vector_size))




  Số lượng ảnh train: 60000
  Chiều dài ảnh train: 28
  Chiều cao ảnh train: 28
  Chiều ảnh được duỗi: 784



### 7.2. Tiền xử lý dữ liệu

Các hàm này được sử dụng từ bài Softmax Regression, nếu bạn chưa hiểu kỹ bài tập này thì hãy làm bài tập đó trước nhé ;)

In [ ]:
def one_hot(y, num_classes):
  """
  Đầu vào:
    y:
      Dạng: numpy array
      Miêu tả: Các giá trị nhãn
      Chiều: (batch_size,)
    num_classes:
      Dạng: Python integer
      Miêu tả: Số lượng nhãn
      Điều kiện: num_classes > 0
      Ví dụ: 10
  Đầu ra:
    y_one_hot:
      Miêu tả: Trả về ma trận các vector one hot của từng nhãn
      Chiều: (batch_size, num_classes)
  """
  y_one_hot = np.squeeze(np.eye(num_classes)[y.reshape(-1)])

  return y_one_hot

def flatten_images(images):
  """
  Thực thi duỗi ảnh từ (batch_size, width, height) thành (batch_size, width x height)
  Đầu vào:
    images:
      Dạng: numpy array
      Miêu tả: Ma trận các ảnh
      Chiều: (batch_size, width, height)
      Ví dụ: (32, 28, 28)
  Đầu ra:
    flattened_images
      Miêu tả: ma trận trong đó các ảnh được duỗi thành vector
      Chiều: (batch_size, image_vector_size) = (batch_size, width x height)
      Ví dụ: (32, 28 x 28) = (32, 784)
  """
  flattened_images = np.reshape(images, (images.shape[0], -1))

  return flattened_images

def normalize_images(images):
  """
  Hàm chuẩn hóa ảnh các giá trị pixel để nằm trong khoảng từ 0 đến 1
  Đầu vào:
    images:
      Dạng: numpy array
      Miêu tả: Ma trận các vector ảnh
      Chiều: (batch_size, image_vector_size)
      Ví dụ: (32, 784)
  Đầu ra:
    normailized_images
      Miêu tả: ma trận trong đó các vector ảnh được chuẩn hóa
      Chiều: (batch_size, image_vector_size)
      Ví dụ: (32, 784)
  """
  normailized_images = images / 255.0

  return normailized_images

try:
  X_train = flatten_images(X_train)
  X_val = flatten_images(X_val)
except Exception as e:
  print("Lỗi thực thi: ", e)

try:
  X_train = normalize_images(X_train)
  X_val = normalize_images(X_val)
except Exception as e:
   print("Lỗi thực thi: ", e)



### 7.3. Định nghĩa chiều các lớp

In [ ]:
n = image_vector_size
d_1 = 1024
d_2 = 256
c = 10
layers_dims = (n, d_1, d_2, c)

### 7.4. Xây dựng mô hình

**TODO 7**: Lập trình hàm tính độ chính xác

Trong trường hợp này trong thân hàm cần chú ý tới chiều


- $n: $ chiều vector ảnh (784)
- $m: $ số ảnh (60000)
- $c: $ số nhãn (10)

Chiều của $\textbf{X} \in \mathbf{R} ^ {n \times m}$
Chiều của $\textbf{Y} \in \mathbf{R} ^ {m \times 1}$


In [ ]:
def cal_acc(parameters, X, Y):
  """
  Tính độ chính xác của mô hình trên toàn bộ tập dữ liệu
    Bước 1: Thực hiện feed forward để lấy kết quả dự đoán
    Bước 2: So sánh kết quả dự đoán với nhãn thật
  Đầu vào:
    X:
      Dạng: numpy array
      Miêu tả: Các vector ảnh
      Chiều: (image_vector_size, Số lượng điểm dữ liệu)
      Ví dụ: (784, 60000)
    Y:
      Dạng: numpy arry
      Miêu tả: Vector của nhãn
      Chiều: (Số lượng điểm dữ liệu,)
      Ví dụ: (60000,)
  Đầu ra:
    acc:
      Dạng: số thực
      Miêu tả: Độ chính xác của mô hình tại thời điểm hiện tại
      Ví dụ: 0.6
  """
  # Lập trình tại đây
  A0 = X
  W1 = parameters['W1']
  b1 = parameters['b1']
  W2 = parameters['W2']
  b2 = parameters['b2']
  W3 = parameters['W3']
  b3 = parameters['b3']

  Z1, linear_cache1 = linear_forward(A0, W1, b1)
  A1 , cache1 = relu(Z1)
  Z2, linear_cache2 = linear_forward(A1, W2, b2)
  A2, cache2 = relu(Z2)
  Z3, linear_cache3 = linear_forward(A2, W3, b3)
  Y_hat, _ = softmax(Z3)



  acc = np.sum(np.equal(np.argmax(Y_hat.T, axis=1), Y)).astype(float) /Y_hat.shape[1]

  return acc

In [ ]:
np.random.seed(42)
parameters = {
  "W1": np.random.randn(d_1, n),
  "b1": np.zeros((d_1, 1)),
  "W2": np.random.randn(d_2, d_1),
  "b2": np.zeros((d_2, 1)),
  "W3": np.random.randn(num_classes, d_2),
  "b3": np.zeros((num_classes, 1)),
}
Y_mock =  np.array([5,2,6, 4])
X_mock = X_mock = X_train[:4].T / 255.0

A0 = X_mock

W1 = parameters['W1']
b1 = parameters['b1']
W2 = parameters['W2']
b2 = parameters['b2']
W3 = parameters['W3']
b3 = parameters['b3']

print(A0.shape)
Z1, linear_cache1 = linear_forward(A0, W1, b1)
A1 , cache1 = relu(Z1)
print(type(A1))
# print(W2)
# print(b2)

Z2, linear_cache2 = linear_forward(A1, W2, b2)
A2, cache2 = relu(Z2)

Z3, linear_cache3 = linear_forward(A2, W3, b3)
Y_hat, _ = softmax(Z3)

print(np.sum(np.equal(np.argmax(Y_hat.T, axis=1), Y_mock)).astype(float) /Y_hat.shape[1])

print(Y_hat)

(784, 4)
<class 'numpy.ndarray'>
0.5
[[9.29711367e-03 7.38316382e-02 4.03639630e-09 1.50279155e-05]
 [2.41390881e-14 2.53258954e-15 7.80768682e-10 6.02454155e-12]
 [2.12651744e-05 6.76163420e-07 6.73914260e-08 6.31118662e-10]
 [1.18491658e-07 6.45446411e-09 6.37007900e-09 1.24777957e-03]
 [1.39606343e-06 6.40513404e-07 4.69792306e-09 1.92316507e-08]
 [9.46367409e-01 2.99568486e-06 1.55753619e-05 4.87522048e-06]
 [4.42514279e-02 9.26148390e-01 9.99979448e-01 3.53920081e-02]
 [2.32213625e-08 4.67525832e-10 7.28124499e-08 9.55196573e-01]
 [6.12380762e-05 1.31157112e-05 7.54209366e-11 8.14371544e-03]
 [8.08919392e-09 2.53650188e-06 4.82012859e-06 8.45430397e-10]]


Test code

In [ ]:
try:
  np.random.seed(42)
  parameters = {
    "W1": np.random.randn(d_1, n),
    "b1": np.zeros((d_1, 1)),
    "W2": np.random.randn(d_2, d_1),
    "b2": np.zeros((d_2, 1)),
    "W3": np.random.randn(num_classes, d_2),
    "b3": np.zeros((num_classes, 1)),
  }

  Y_mock =  np.array([5,2,6, 4])
  X_mock = X_mock = X_train[:4].T / 255.0
  acc_mock = cal_acc(parameters, X_mock, Y_mock)
  print("Độ chính xác: {:.3f}".format(acc_mock))
except Exception as e:
  print("Lỗi thực thi: ", e)

Độ chính xác: 0.500


**Kết quả mong đợi**:

```
Độ chính xác: 0.500
```

<img src="https://storage.googleapis.com/protonx-cloud-storage/images/Backprop4.PNG"  />

Với 2 lớp ta sẽ có các công thức sau.


**Chiều Backward:**

- Lớp Softmax cuối cùng:

$$\textbf{E}^{(3)} = \hat{\textbf{Y}} - \textbf{Y} $$

$$ d\textbf{A}^{(2)} = \textbf{W}^{(3)T}\textbf{E}^{(3)} $$


$$ d\textbf{W}^{(3)} = \frac{1}{m} \textbf{E}^{(3)}\textbf{A}^{(2)T} \rightarrow \textbf{W}^{(3)} := \textbf{W}^{(3)}  - \alpha \frac{1}{m} \textbf{E}^{(3)}\textbf{A}^{(2)T}    $$

$$ d\textbf{b}^{(3)} = \frac{1}{m} \sum_{i=1}^{m}\textbf{E}^{(3)(i)}\rightarrow \textbf{b}^{(3)} := \textbf{b}^{(3)}  - \alpha \frac{1}{m} \sum_{i=1}^{m}\textbf{E}^{(3)(i)}$$

- Lớp ẩn số 2:

$$\textbf{E}^{(2)} = d\textbf{Z}^{(2)} = {d\textbf{A}^{(2)}} \odot f'(\textbf{Z}^{(2)}) $$

$$ d\textbf{A}^{(1)} = \textbf{W}^{(2)T}\textbf{E}^{(2)} $$

$$ d\textbf{W}^{(2)} = \frac{1}{m} \textbf{E}^{(2)}\textbf{A}^{(1)T} \rightarrow \textbf{W}^{(2)} := \textbf{W}^{(2)}  - \alpha \frac{1}{m} \textbf{E}^{(2)}\textbf{A}^{(1)T}    $$

$$ d\textbf{b}^{(2)} = \frac{1}{m} \sum_{i=1}^{m}\textbf{E}^{(2)(i)}\rightarrow \textbf{b}^{(2)} := \textbf{b}^{(2)}  - \alpha \frac{1}{m} \sum_{i=1}^{m}\textbf{E}^{(2)(i)}$$


- Lớp ẩn số 1:

$$\textbf{E}^{(1)} = d\textbf{Z}^{(1)} = {d\textbf{A}^{(1)}} \odot f'(\textbf{Z}^{(1)}) $$

$$ d\textbf{W}^{(1)} = \frac{1}{m} \textbf{E}^{(1)}\textbf{X}^{T} \rightarrow \textbf{W}^{(1)} := \textbf{W}^{(1)}  - \alpha \frac{1}{m} \textbf{E}^{(1)}\textbf{X}^{T}    $$

$$ d\textbf{b}^{(1)} = \frac{1}{m} \sum_{i=1}^{m}\textbf{E}^{(1)(i)}\rightarrow \textbf{b}^{(1)} := \textbf{b}^{(1)}  - \alpha \frac{1}{m} \sum_{i=1}^{m}\textbf{E}^{(1)(i)}$$




Nếu bạn thắc mắc tại sao $$\textbf{E}^{(3)} = \hat{\textbf{Y}} - \textbf{Y} $$

thì lời giải nằm [tại đây](https://colab.research.google.com/drive/1awivtxQzjgNK2m-cEQLIyzE5g6MxcZpX#scrollTo=NYrcc6YWGz1S).

**TODO 8:** Xây dựng hàm train model

In [ ]:
def train_model(X_train, Y_train, X_val, Y_val, layers_dims, learning_rate=0.1, epochs=250):
    """
    Hàm xây dựng và train model
      Bước 1: Thực hiện lan truyền thuận, tính giá trị đầu ra của các lớp
      Bước 2: Tính giá trị mất mát
      Bước 3: Thực hiện lan truyền ngược, tính giá trị đạo hàm của hàm mất mát trên các tham số
      Bước 4: Cập nhật tham số của mô hình
    Đầu vào:
      X_train:
        Dạng: numpy array
        Miêu tả: Các vector ảnh
        Chiều: (Số lượng điểm dữ liệu, image_vector_size)
        Ví dụ: (60000, 784)
      Y_train:
        Dạng: numpy arry
        Miêu tả: Vector của nhãn
        Chiều: (Số lượng điểm dữ liệu,)
        Ví dụ: (60000,)
      X_val:
        Dạng: numpy array
        Miêu tả: Các vector ảnh
        Chiều: (Số lượng điểm dữ liệu, image_vector_size)
        Ví dụ: (10000, 784)
      Y_val:
        Dạng: numpy arry
        Miêu tả: Vector của nhãn
        Chiều: (Số lượng điểm dữ liệu,)
        Ví dụ: (10000,)
      layers_dims:
        Dạng: Python tuple
        Miêu tả: (n, d_1, d_2, num_classes)
          n: Chiều của vector ảnh
          d_1: Chiều của lớp ẩn 1
          d_2: Chiều của lớp ẩn 2
          num_classes: số nhãn
      learning_rate:
        Dạng: Python float
        Miêu tả: Tốc độ học của mô hình
        Ví dụ: 0.001
      epochs:
        Dạng: Python integer
        Miêu tả: Số vòng lặp qua tập dữ liệu
        Ví dụ: 100
    Đầu ra:
      Python Tuple: (parameters, acc, val_acc, costs)
      parameters:
        Dạng: Python Dictionary
        Miêu tả: Bộ tham số của mô hình
      acc:
        Dạng: số thực
        Miêu tả: Độ chính xác của mô hình trên bộ train
        Ví dụ: 0.6
      val_acc:
        Dạng: số thực
        Miêu tả: Độ chính xác của mô hình trên bộ validation
        Ví dụ: 0.7
      costs:
        Dạng: Python List
        Miêu tả: Giá trị mất mát trên từng epoch
        Ví dụ [0.2, 0.15, 0.1]
    """

    np.random.seed(42)
    grads = {}
    costs = []
    m = X_train.shape[0]
    (n, d_1, d_2, num_classes) = layers_dims

    # 0. Khởi tạo tham số

    parameters = initialize_parameters(n, d_1, d_2, num_classes)

    # 1. Điều chỉnh chiều dữ liệu sao cho
    # X_train: (image_vector_size, Số lượng điểm dữ liệu)
    # Y_train_one_hot: (Số lượng nhãn, Số lượng điểm dữ liệu)
    # X_val: (image_vector_size, Số lượng điểm dữ liệu)

    X_train = X_train.T
    Y_train_one_hot = one_hot(Y_train, num_classes).T
    X_val = X_val.T
    Y_val_one_hot = one_hot(Y_val, num_classes).T


    # 2. Lấy các tham số ra khỏi dictionary: parameters
    A0 = X_train
    # W1 = parameters['W1']
    # b1 = parameters['b1']
    # W2 = parameters['W2']
    # b2 = parameters['b2']
    # W3 = parameters['W3']
    # b3 = parameters['b3']

    # 3. Lấy các tham số ra khỏi dictionary: parameters
    for i in range(0, epochs):


        # 3.1. Thực hiện Feed Forward trên 3 lớp: 2 lớp relu và lớp Softmax


        # Forward pass
        print('start 3.1')
        Z1, linear_cache1 = linear_forward(X_train, parameters['W1'], parameters['b1'])
        A1, cache1 = relu(Z1)
        Z2, linear_cache2 = linear_forward(A1, parameters['W2'], parameters['b2'])
        A2, cache2 = relu(Z2)
        Z3, linear_cache3 = linear_forward(A2, parameters['W3'], parameters['b3'])
        Y_hat, cache3 = softmax(Z3)


        # 3.2. Tính giá trị mất mát
        print('start 3.2')
        cost = compute_cost(Y_hat, Y_train_one_hot)
        print(cost)

        # 3.3. Tính giá trị E3
        print('start 3.3')
        E3 = Y_hat - Y_train_one_hot


        # 3.4. Tính giá trị dA2
        print('start 3.4')

        dA2 = np.dot(parameters['W3'].T, E3)


        # 3.5. Tính giá trị dW3
        dW3 = np.dot(E3, A2.T) / m




        # 3.6. Tính giá trị db3
        db3 = np.sum(E3, axis=1, keepdims=True) / m

        # Tính Accuracy


        # 3.7. Tính độ chính xác trên tập train
        print('start 3.7')
        acc = cal_acc(parameters, X_train, Y_train)
        print(acc)
        # Y_hat

        # 3.8. Tính độ chính xác trên tập validation
        print('start 3.8')
        val_acc = cal_acc(parameters, X_val, Y_val)

        # 3.9. Thực hiện backward trên lớp số 2 để tính ra dA1, dW2, db2
        print('start 3.9')
        E2 = relu_backward(dA2, Z2)
        dW2 = np.dot(E2, A1.T) / m
        db2 = np.sum(E2, axis=1, keepdims=True) / m
        dA1 = np.dot(parameters['W2'].T, E2)



        # 3.10. Thực hiện backward trên lớp số 1 để tính ra dA0, dW1, db1
        print('start 3.10')
        E1 = relu_backward(dA1, Z1)
        dW1 = np.dot(E1, X_train.T) / m
        db1 = np.sum(E1, axis=1, keepdims=True) / m
        dA0 = np.dot(parameters['W1'].T, E1)
        # dA1, dW1, db1 = linear_backward(E1, linear_cache1)

        # 3.11. Cập nhật các giá trị dW1, db1, dW2, db2, dW3, db3 vào dictionary grads
        # print('start 3.11')
        grads = {
          "dW1": dW1,
          "db1": db1,
          "dW2": dW2,
          "db2": db2,
          "dW3": dW3,
          "db3": db3,
        }
        # print(grads)


        # 3.12. Tiến hành cập nhật các tham số của mô hình
        # update_parameters(parameters, grads, learning_rate)
          parameters = update_parameters(parameters, grads, learning_rate)
        # print('W1 before', W1)
        # W1 = parameters['W1'] - learning_rate * grads['dW1']
        # print('W1 after', W1)
        # b1 = parameters['b1'] - learning_rate * grads['db1']

        # W2 = parameters['W2'] - learning_rate * grads['dW2']
        # b2 = parameters['b2'] - learning_rate * grads['db2']

        # W3 = parameters['W3'] - learning_rate * grads['dW3']
        # b3 = parameters['b3'] - learning_rate * grads['db3']


        # 3.13. Gán các giá trị tham số mới vào các biến đã được định nghĩa sẵn
        # update_parameters(parameters, grads, learning_rate)
        # parameters = {'W1': W1, 'b1': b1, 'W2': W2, 'b2':b2, 'W3': W3, 'b3':b3}


        print("Epoch {}: Cost: {:.3f} Acc: {:.3f} Validation Acc: {:.3f}".format(i + 1, np.squeeze(cost), acc, val_acc))

        costs.append(cost)


    return parameters, acc, val_acc, costs

In [ ]:
# (X_train, Y_train), (X_val, Y_val) = tf.keras.datasets.mnist.load_data()

# try:
#   X_train = flatten_images(X_train)
#   X_val = flatten_images(X_val)
# except Exception as e:
#   print("Lỗi thực thi: ", e)

# try:
#   X_train = normalize_images(X_train)
#   X_val = normalize_images(X_val)
# except Exception as e:
#    print("Lỗi thực thi: ", e)


# np.random.seed(42)
# grads = {}
# costs = []
# m = X_train.shape[0]
# (n, d_1, d_2, num_classes) = layers_dims

# parameters = initialize_parameters(n, d_1, d_2, num_classes)
# X_train = X_train.T
# Y_train_one_hot = one_hot(Y_train, num_classes).T
# X_val = X_val.T

# A0 = X_train
# W1 = parameters['W1']
# b1 = parameters['b1']
# W2 = parameters['W2']
# b2 = parameters['b2']
# W3 = parameters['W3']
# b3 = parameters['b3']

# Z0, linear_cache = linear_forward(A0, W1, b1)
# A1 , cache1 = relu(Z0)
# Z1, linear_cache = linear_forward(A1, W2, b2)
# A2, cache2 = relu(Z1)
# Z2, linear_cache = linear_forward(A2, W3, b3)
# Y_hat, _ = softmax(Z2)

# Y_hat = Y_hat.T
# print(Y_train_one_hot, Y_train_one_hot.shape)
# cal_acc(parameters, X_train/ 255.0, Y_train)

In [ ]:
# Y_hat.T - Y_train_one_hot
# Y_train_one_hot[:1].shape

### 7.5. Tiến hành training

**TODO 9**: Tiến hành training

In [ ]:
try:
  # plot the cost
  epochs = 400
  learning_rate = 0.1
  parameters, acc, val_acc, costs  = train_model(X_train, Y_train, X_val, Y_val, layers_dims = layers_dims, learning_rate=learning_rate, epochs=epochs)
except Exception as e:
  print("Lỗi thực thi: ", e)

start 3.1
start 3.2
2.3025220583041204
start 3.3
start 3.4
start 3.7
0.08628333333333334
start 3.8
start 3.9
start 3.10
Epoch 1: Cost: 2.303 Acc: 0.086 Validation Acc: 0.081
start 3.1
start 3.2
2.302351677442841
start 3.3
start 3.4
start 3.7
0.09166666666666666
start 3.8
start 3.9
start 3.10
Epoch 2: Cost: 2.302 Acc: 0.092 Validation Acc: 0.085
start 3.1
start 3.2
2.3021809901691124
start 3.3
start 3.4
start 3.7
0.12276666666666666
start 3.8
start 3.9
start 3.10
Epoch 3: Cost: 2.302 Acc: 0.123 Validation Acc: 0.117
start 3.1
start 3.2
2.3020097881166195
start 3.3
start 3.4
start 3.7
0.17776666666666666
start 3.8
start 3.9
start 3.10
Epoch 4: Cost: 2.302 Acc: 0.178 Validation Acc: 0.174
start 3.1
start 3.2
2.301837921084806
start 3.3
start 3.4
start 3.7
0.19065
start 3.8
start 3.9
start 3.10
Epoch 5: Cost: 2.302 Acc: 0.191 Validation Acc: 0.190
start 3.1
start 3.2
2.3016651526003535
start 3.3
start 3.4
start 3.7
0.18863333333333332
start 3.8
start 3.9
start 3.10
Epoch 6: Cost: 2.302 Acc

In [ ]:
# trained_parameters = {}
# if not trained_parameters:
#   print('yes')
parameters

{'W1': array([[ 0.00496714, -0.00138264,  0.00647689, ..., -0.01335344,
          0.00380198,  0.00610586],
        [ 0.0055979 ,  0.01080781,  0.00833922, ..., -0.00281328,
          0.00066991,  0.00515939],
        [-0.01562546, -0.00529053,  0.00794265, ..., -0.00392013,
          0.01059936,  0.00617006],
        ...,
        [-0.01089676,  0.02289327, -0.00581598, ...,  0.00013943,
         -0.01802024,  0.01177402],
        [-0.02091988, -0.01580514,  0.0165381 , ..., -0.00200309,
          0.00430815,  0.00156364],
        [-0.01364376,  0.00343994, -0.01403063, ...,  0.00351143,
          0.01729455,  0.01028495]]),
 'b1': array([[0.00221674],
        [0.01986265],
        [0.00314107],
        ...,
        [0.00268051],
        [0.00942159],
        [0.01426119]]),
 'W2': array([[ 0.01352095, -0.00188469,  0.01250039, ...,  0.02185181,
         -0.00675145,  0.01736718],
        [ 0.00476641, -0.01652994, -0.00624038, ..., -0.01474519,
         -0.00520355, -0.00339916],
    

**Kết quả mong đợi (Tương đối)**:

```
Epoch 387: Cost: 0.421 Acc: 0.880 Validation Acc: 0.882
Epoch 388: Cost: 0.420 Acc: 0.881 Validation Acc: 0.883
Epoch 389: Cost: 0.420 Acc: 0.881 Validation Acc: 0.883
Epoch 390: Cost: 0.419 Acc: 0.881 Validation Acc: 0.883
Epoch 391: Cost: 0.418 Acc: 0.881 Validation Acc: 0.884
Epoch 392: Cost: 0.418 Acc: 0.881 Validation Acc: 0.884
Epoch 393: Cost: 0.417 Acc: 0.882 Validation Acc: 0.884
Epoch 394: Cost: 0.416 Acc: 0.882 Validation Acc: 0.884
Epoch 395: Cost: 0.416 Acc: 0.882 Validation Acc: 0.884
Epoch 396: Cost: 0.415 Acc: 0.882 Validation Acc: 0.884
Epoch 397: Cost: 0.414 Acc: 0.882 Validation Acc: 0.884
Epoch 398: Cost: 0.414 Acc: 0.882 Validation Acc: 0.884
Epoch 399: Cost: 0.413 Acc: 0.883 Validation Acc: 0.884
Epoch 400: Cost: 0.412 Acc: 0.883 Validation Acc: 0.885
```

Chúc mừng bạn đã vượt qua một thử thách lớn trong con đường học AI của mình.

<img src="https://storage.googleapis.com/protonx-cloud-storage/icons/488-bicycle-outline.gif" />